# Variational Quantum Eigensolver

The algorithm employs a variational quantum circuit to find the smallest eigenvalue of an operator (e.g. The hamiltonian of the system trying to simulate). 
In case the simulated system is a molecule, then the smallest eigenvalue of the Hamiltonian operator is the ground state energy of the quantum system

Let $\psi(\theta)$ be the quantum state produced by the variational circuit. The classical optimization tries to minimize the the expectation value of the Hamiltonial operator (as cost function)

$ \lambda_{min} \le \lambda_{\theta} = <\psi(\theta),H\psi(\theta)> $

In the case of VQE, the optimization chois will be the SPSA (Simultaneous Perturbation Stochastic Approximation optimizer) which is less consuming in computational terms and more preferable for NISQ era devices. Also in case of simulation (or fault tollerant devices when they will be available) where there is no noise in the system, one can try other classical optimizers (e.g. Sequential Least Squares Programming optimizer (SLSQP) and the Constrained Optimization by Linear Approximation optimizer (COBYLA)).

In [91]:
import numpy as np
from qiskit import QuantumCircuit, execute
from qiskit.visualization import plot_histogram

In [92]:
from qiskit.circuit.library import EfficientSU2
from qiskit import Aer

In [93]:
simulator = simulator = Aer.get_backend('qasm_simulator')

np.random.seed(999999)
target_vec = np.random.rand(2)
target_vec /= sum(target_vec)

def get_variation_circuit(params) :
    circuit = QuantumCircuit(1, 1)

    circuit.u3(params[0], params[1], params[2], 0)
    circuit.measure(0, 0)

    return circuit

def cost_function(params): 
    circuit = get_variation_circuit(params)

    result = execute(circuit, backend = simulator, shots = 1000).result()

    # get the probabilities from the shots
    probabilities = [value / 1000 for value in result.get_counts(circuit).values()]

    cost = 0

    cost = sum([np.abs(probabilities[i]) - target_vec[i] for i in range(2)])
    return cost

In [95]:
from qiskit.algorithms.optimizers import COBYLA

classical_optimizer = COBYLA(maxiter = 500, tol = 0.0001)

initial_params = np.random.rand(3)

results = classical_optimizer.optimize(num_vars = 3, objective_function = cost_function, initial_point = initial_params)

print("Parameters found: ", results[0])

# get the last instance of the parameters and assign them to the variational circuit
circuit = get_variation_circuit(results[0])

result = execute(circuit, backend = simulator, shots = 1000).result()

# get the probabilities from the shots
final_probabilities = [value / 1000 for value in result.get_counts(circuit).values()]

print("Target Distribution:", target_vec)
print("Obtained Distribution:", final_probabilities)

Parameters found:  [0.65780185 0.20156804 0.6716189 ]
Target Distribution: [0.51357006 0.48642994]
Obtained Distribution: [0.107, 0.893]


/tmp/ipykernel_17833/400659625.py:13: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  circuit.u3(params[0], params[1], params[2], 0)
